# Cohere Health Machine Learning Takehome Assignment
The following notebook was generated by members of the Cohere Health Machine Learning team and should be used for the purpose of progressing the interview takehome assignment. The code contained within this notebook is merely an example of how candidates can get started and is optional to use. Feel free to reach out to your recruiting team if you have any questions.

## Load in Data
You must provide YOUR OWN PATH to the location of the sampleclinicalnotes.zip file in the `PATH_TO_ZIP` object.

In [1]:
from load_data import load_ann, load_txt
import pandas as pd
import text_utils
import df_utils

In [2]:
PATH_TO_ZIP = "data/"
DATA_PATH = f"{PATH_TO_ZIP}training_20180910/"
print(f"Full data path: {DATA_PATH}")

Full data path: data/training_20180910/


In [3]:
# read in txt files
txt_df = load_txt(DATA_PATH)

Time taken to read .txt files: 0.03354072570800781


In [4]:
# read in REASONS entities from .ann files
ent_df, rel_df = load_ann(DATA_PATH)

Time taken to read .ann files and extract all metadata: 0.1481337547302246


In [5]:
all_dfs = [txt_df, ent_df, rel_df]
txt_df, ent_df, rel_df = [df_utils.fix_slashes(df) for df in all_dfs]

In [6]:
rel_df = df_utils.clean_rel_df(rel_df)
ent_df = df_utils.clean_ent_df(ent_df)

In [7]:
txt_df.head()

,file_idx,text
0,training_20180910/100035,Admission Date: [**2115-2-22**] ...
1,training_20180910/100039,Admission Date: [**2174-4-18**] ...
2,training_20180910/100187,Admission Date: [**2107-1-17**] ...
3,training_20180910/100229,Admission Date: [**2114-12-24**] ...
4,training_20180910/100564,Admission Date: [**2144-1-20**] ...


In [8]:
ent_df.head()

,file_idx,entity_id,category,start_idx,end_idx,text
0,training_20180910/100035,T1,Reason,10179,10197,recurrent seizures
1,training_20180910/100035,T3,Drug,10227,10233,ativan
2,training_20180910/100035,T5,Route,10240,10242,IM
3,training_20180910/100035,T6,Drug,10455,10465,Topiramate
4,training_20180910/100035,T7,Strength,10466,10470,25mg


In [9]:
rel_df.head()

,file_idx,relationship_id,category,entity1,entity2
0,training_20180910/100035,R1,Reason-Drug,T1,T3
1,training_20180910/100035,R4,Route-Drug,T5,T3
2,training_20180910/100035,R5,Strength-Drug,T7,T6
3,training_20180910/100035,R6,Route-Drug,T8,T6
4,training_20180910/100035,R7,Frequency-Drug,T9,T6


In [10]:
rel_df.category.value_counts()

category
Strength-Drug     6702
Form-Drug         6654
Frequency-Drug    6310
Route-Drug        5538
Reason-Drug       5169
Dosage-Drug       4225
ADE-Drug          1107
Duration-Drug      643
Name: count, dtype: int64

In [11]:
ent_df['length_diff'] = (ent_df.end_idx - ent_df.start_idx) - ent_df.text.str.len()

In [12]:
ent_df[(ent_df.category == 'Reason') & (ent_df.length_diff != 0)]

,file_idx,entity_id,category,start_idx,end_idx,text,length_diff
271,training_20180910/100035,T352,Reason,9314,9324,grand mal seizure,-7
581,training_20180910/100039,T294,Reason,25084,25092,prevent infection,-9
585,training_20180910/100039,T298,Reason,25152,25162,uric acid levels were high,-16
957,training_20180910/100187,T355,Reason,7425,7436,multifocal pneumonia,-9
984,training_20180910/100187,T382,Reason,8535,8553,breaking up thick sputum,-6
...,...,...,...,...,...,...,...
50715,training_20180910/196798,T368,Reason,14901,14911,continued outbursts,-9
50741,training_20180910/196798,T400,Reason,22979,22991,transiently febrile,-7
50746,training_20180910/196798,T405,Reason,23409,23418,Bacillus species grew from [**6-19**] PICC blo...,-42
50751,training_20180910/196798,T412,Reason,25470,25476,yeast infection,-9


In [13]:
txt_df[txt_df['file_idx'] == 'training_20180910/198406'].text.values[0][12981:12985+11]

'DVT\nprophylaxis'

In [14]:
txt_df = df_utils.split_txt_df(txt_df)

In [15]:
print(txt_df.primary_diagnosis[301])

Lymphedema with superimposed cellulitis and unremitting pain


In [16]:
ent_df[(ent_df.category == 'Reason') & (ent_df.length_diff != 0)]

,file_idx,entity_id,category,start_idx,end_idx,text,length_diff
271,training_20180910/100035,T352,Reason,9314,9324,grand mal seizure,-7
581,training_20180910/100039,T294,Reason,25084,25092,prevent infection,-9
585,training_20180910/100039,T298,Reason,25152,25162,uric acid levels were high,-16
957,training_20180910/100187,T355,Reason,7425,7436,multifocal pneumonia,-9
984,training_20180910/100187,T382,Reason,8535,8553,breaking up thick sputum,-6
...,...,...,...,...,...,...,...
50715,training_20180910/196798,T368,Reason,14901,14911,continued outbursts,-9
50741,training_20180910/196798,T400,Reason,22979,22991,transiently febrile,-7
50746,training_20180910/196798,T405,Reason,23409,23418,Bacillus species grew from [**6-19**] PICC blo...,-42
50751,training_20180910/196798,T412,Reason,25470,25476,yeast infection,-9


In [17]:
txt_df.history[0]

'Mr. [**Known lastname 3234**] is a 36 year old gentleman with a PMH signifciant with dilated cardiomyopathy s/p AICD, asthma, and HTN admitted to an OSH with dyspnea now admitted to the MICU after PEA arrest x2. The patient initially presented to LGH ED with hypoxemic respiratory distress. While at the OSH, he received CTX, azithromycin, SC epinephrine, and solumedrol. While at the OSH, he became confused and subsequently had an episode of PEA arrest and was intubated. He received epinephrine, atropine, magnesium, and bicarb. In addition, he had bilateral needle thoracostomies with report of air return on the left, and he subsequently had bilateral chest tubes placed.  After approximately 15-20 minutes of rescucitation, he had ROSC. He received vecuronium and was started on an epi gtt for asthma and a cooling protocol, and was then transferred to [**Hospital1 18**] for further evaluation. Of note, the patient was admitted to LGH in [**1-4**] for dyspnea, and was subsequently diagnosed

In [18]:
combo_df = txt_df.join(ent_df[ent_df.category == 'Reason'].set_index('file_idx'), on='file_idx', lsuffix='_txt', rsuffix='_ent', validate='1:m')

In [21]:
full_df = df_utils.get_underlying_factors(txt_df, ent_df)

In [25]:
full_df[['primary_diagnosis', 'underlying_factors']][full_df.underlying_factors.notna()]

,primary_diagnosis,underlying_factors
0,Anoxic Brain Injury s/p PEA arrest x,"seizures, seizure, asthma"
1,Abdominal Pain,nausea
2,Pulmonary Embolism with history of DVT and IVC...,"anticoagulation, multifocal pneumonia, pneumon..."
3,Sepsis,"hyperkalemia, renal transplant, infection, adr..."
4,Deep Vein Thrombosis of subclavian vein,"thrombolysis, pain"
...,...,...
293,Extensive liver lesions (presumably recurrent ...,"right upper quadrant pain, abdominal pain"
294,Metastatic anaplastic thyroid cancer,stridor
296,site,"a-fib, atrial fibrillation"
297,Coronary Artery Disease,"hypotensive, chest pain, ventricular tachycardia"
